In [1]:
import os
import sys
import requests

import numpy as np
import pandas as pd

sys.path.insert(0, '..')
from model.BasicModel import BasicModel


In [2]:
df_path = os.path.join(os.getcwd(), os.pardir, 'data', 'data.csv')
df = pd.read_csv(df_path)
df.head()

,acousticness,artists,danceability,duration_ms,energy,explicit,id,instrumentalness,key,liveness,loudness,mode,name,popularity,release_date,speechiness,tempo,valence,year
0,0.995,['Carl Woitschach'],0.708,158648,0.1950,0,6KbQ3uYMLKb5jDxLF7wYDD,0.563,10,0.1510,-12.428,1,Singende Bataillone 1. Teil,0,1928,0.0506,118.469,0.7790,1928
1,0.994,"['Robert Schumann', 'Vladimir Horowitz']",0.379,282133,0.0135,0,6KuQTIu1KoTTkLXKrwlLPV,0.901,8,0.0763,-28.454,1,"Fantasiestücke, Op. 111: Più tosto lento",0,1928,0.0462,83.972,0.0767,1928
2,0.604,['Seweryn Goszczyński'],0.749,104300,0.2200,0,6L63VW0PibdM1HDSBoqnoM,0.000,5,0.1190,-19.924,0,Chapter 1.18 - Zamek kaniowski,0,1928,0.9290,107.177,0.8800,1928
3,0.995,['Francisco Canaro'],0.781,180760,0.1300,0,6M94FkXd15sOAOQYRnWPN8,0.887,1,0.1110,-14.734,0,Bebamos Juntos - Instrumental (Remasterizado),0,1928-09-25,0.0926,108.003,0.7200,1928
4,0.990,"['Frédéric Chopin', 'Vladimir Horowitz']",0.210,687733,0.2040,0,6N6tiFZ9vLTSOIxkj8qKrd,0.908,11,0.0980,-16.829,1,"Polonaise-Fantaisie in A-Flat Major, Op. 61",1,1928,0.0424,62.149,0.0693,1928


In [3]:
df_train = df[['acousticness', 'danceability','energy',
               'instrumentalness', 'key', 'liveness', 'loudness',
               'mode','speechiness', 'tempo',
               'valence']]

In [4]:
df_train.dtypes

acousticness        float64
danceability        float64
energy              float64
instrumentalness    float64
key                   int64
liveness            float64
loudness            float64
mode                  int64
speechiness         float64
tempo               float64
valence             float64
dtype: object

In [5]:
df_train.head()

,acousticness,danceability,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,valence
0,0.995,0.708,0.1950,0.563,10,0.1510,-12.428,1,0.0506,118.469,0.7790
1,0.994,0.379,0.0135,0.901,8,0.0763,-28.454,1,0.0462,83.972,0.0767
2,0.604,0.749,0.2200,0.000,5,0.1190,-19.924,0,0.9290,107.177,0.8800
3,0.995,0.781,0.1300,0.887,1,0.1110,-14.734,0,0.0926,108.003,0.7200
4,0.990,0.210,0.2040,0.908,11,0.0980,-16.829,1,0.0424,62.149,0.0693


In [6]:
model = BasicModel()

In [7]:
model.build(10)

In [8]:
model.fit(df_train)

NearestNeighbors(n_neighbors=10)

In [9]:
test = np.array(df_train.iloc[20215])
test

array([ 2.03000e-01,  6.09000e-01,  4.43000e-01,  1.04000e-03,
        1.10000e+01,  1.83000e-01, -1.14780e+01,  0.00000e+00,
        3.05000e-02,  1.22792e+02,  2.11000e-01])

In [10]:
test = test.reshape(1, -1)
scores, indices = model.predict(test)

In [11]:
results = []
for i, index in enumerate(indices[0]):
    track_id = df.iloc[index]['id']
    artists = df.iloc[index]['artists']
    title = df.iloc[index]['name']
    results.append({'index': index,
                    'track_id': track_id,
                    'artists': artists,
                    'title': title,
                    'score': scores[0][i]})
    
result_table = pd.DataFrame(results)
result_table.sort_values(by='score')

,index,track_id,artists,title,score
0,20215,4nTXzIW8EjH0V1NBxyhatX,['Grateful Dead'],Lost Sailor - 2013 Remaster,0.000000
1,104025,4nheObMzE4K8AIxIICoLgd,['Nacha Pop'],Lucha De Gigantes,0.699675
2,39635,49ONd7q61KYjSZE0A8gtCW,['K CAMP'],Blessing,0.715049
3,59645,2Zfo8pTcTuxMlzcNZhtfUC,['Nick Cave & The Bad Seeds'],From Her to Eternity - 2009 Remastered Version,0.780426
4,125668,7g3htkaLz4ETFn0cifwM3y,['Lana Del Rey'],High By The Beach,0.789610
5,27950,2eH6hxg55caYvT8tWqoaTf,['kalapana'],(For You) I'd Chase a Rainbow,0.805592
6,65168,3esvklLKEej56kihIzxMnn,['The Rolling Stones'],Play With Fire - Mono,0.822362
7,31545,1RgiH5ucgoEWDKmGyA5drQ,['Sleeping At Last'],Neptune,0.834309
8,103420,0ehVk4Py6iyLpfruxVYq4S,['Stevie Nicks'],Kind of Woman - 2016 Remaster,0.845039
9,93545,5BUq9bdtsm3LWw5laJeJ4R,['Olivia Newton-John'],Hopelessly Devoted To You - From “Grease”,0.874753
